In [1]:
import pandas as pd

In [2]:
# 读取CSV文件
df = pd.read_csv('test.csv')
# 打印数据
print(df)

                    DateTime Global_active_power Global_reactive_power  \
0        2009-01-01 00:00:00               0.484                 0.062   
1        2009-01-01 00:01:00               0.484                 0.062   
2        2009-01-01 00:02:00               0.482                 0.062   
3        2009-01-01 00:03:00               0.482                  0.06   
4        2009-01-01 00:04:00               0.482                 0.062   
...                      ...                 ...                   ...   
1000618  2010-11-26 20:58:00               0.946                   0.0   
1000619  2010-11-26 20:59:00               0.944                   0.0   
1000620  2010-11-26 21:00:00               0.938                   0.0   
1000621  2010-11-26 21:01:00               0.934                   0.0   
1000622  2010-11-26 21:02:00               0.932                   0.0   

        Voltage Global_intensity Sub_metering_1 Sub_metering_2  \
0        247.86              2.2            0

/tmp/ipykernel_632161/1424781712.py:2: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/test.csv')


In [3]:
df.dtypes

DateTime                  object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
RR                       float64
NBJRR1                   float64
NBJRR5                   float64
NBJRR10                  float64
NBJBROU                  float64
dtype: object

In [4]:
def to_float(df,name):
    df[name] = pd.to_numeric(df[name], errors='coerce')
    df[name] = df[name].fillna(0)

In [5]:
to_float(df,'Global_active_power')
to_float(df,'Global_reactive_power')
to_float(df,'Voltage')
to_float(df,'Global_intensity')
to_float(df,'Sub_metering_1')
to_float(df,'Sub_metering_2')
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [6]:
df.dtypes

DateTime                 datetime64[ns]
Global_active_power             float64
Global_reactive_power           float64
Voltage                         float64
Global_intensity                float64
Sub_metering_1                  float64
Sub_metering_2                  float64
Sub_metering_3                  float64
RR                              float64
NBJRR1                          float64
NBJRR5                          float64
NBJRR10                         float64
NBJBROU                         float64
dtype: object

In [7]:
df2 = df.copy()
df2.set_index('DateTime', inplace=True)
print(df2)

                     Global_active_power  Global_reactive_power  Voltage  \
DateTime                                                                   
2009-01-01 00:00:00                0.484                  0.062   247.86   
2009-01-01 00:01:00                0.484                  0.062   247.72   
2009-01-01 00:02:00                0.482                  0.062   247.75   
2009-01-01 00:03:00                0.482                  0.060   247.52   
2009-01-01 00:04:00                0.482                  0.062   246.94   
...                                  ...                    ...      ...   
2010-11-26 20:58:00                0.946                  0.000   240.43   
2010-11-26 20:59:00                0.944                  0.000   240.00   
2010-11-26 21:00:00                0.938                  0.000   239.82   
2010-11-26 21:01:00                0.934                  0.000   239.70   
2010-11-26 21:02:00                0.932                  0.000   239.55   

           

In [8]:
daily_df = df2.resample('D').agg(
    Global_active_power=pd.NamedAgg(column='Global_active_power', aggfunc='sum'), 
    Global_reactive_power=pd.NamedAgg(column='Global_reactive_power', aggfunc='sum'), 
    Voltage=pd.NamedAgg(column='Voltage', aggfunc='mean'), 
    Global_intensity=pd.NamedAgg(column='Global_intensity', aggfunc='mean'),
    Sub_metering_1=pd.NamedAgg(column='Sub_metering_1', aggfunc='sum'), 
    Sub_metering_2=pd.NamedAgg(column='Sub_metering_2', aggfunc='sum'),
    Sub_metering_3=pd.NamedAgg(column='Sub_metering_3', aggfunc='sum'),
    RR=pd.NamedAgg(column='RR', aggfunc='last'), 
    NBJRR1=pd.NamedAgg(column='NBJRR1', aggfunc='last'),
    NBJRR5=pd.NamedAgg(column='NBJRR5', aggfunc='last'),
    NBJRR10=pd.NamedAgg(column='NBJRR10', aggfunc='last'),
    NBJBROU=pd.NamedAgg(column='NBJBROU', aggfunc='last')
)
daily_df = daily_df.reset_index()
print(daily_df)

      DateTime  Global_active_power  Global_reactive_power     Voltage  \
0   2009-01-01             1406.692                112.668  242.836062   
1   2009-01-02             2327.510                204.730  242.704889   
2   2009-01-03             1913.496                174.912  243.681458   
3   2009-01-04             1939.994                167.338  244.822431   
4   2009-01-05             1604.000                128.938  244.830778   
..         ...                  ...                    ...         ...   
690 2010-11-22             2041.536                142.354  240.197118   
691 2010-11-23             1577.536                137.450  240.575528   
692 2010-11-24             1796.248                132.460  240.030965   
693 2010-11-25             1431.164                116.128  241.536257   
694 2010-11-26             1488.104                120.826  240.291029   

     Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3     RR  \
0            4.149861          

In [9]:
daily_df['Sub_metering_4'] = (daily_df['Global_active_power'] * 1000 / 60 
                              - daily_df['Sub_metering_1'] 
                              - daily_df['Sub_metering_2'] 
                              - daily_df['Sub_metering_3'])
print(daily_df)

      DateTime  Global_active_power  Global_reactive_power     Voltage  \
0   2009-01-01             1406.692                112.668  242.836062   
1   2009-01-02             2327.510                204.730  242.704889   
2   2009-01-03             1913.496                174.912  243.681458   
3   2009-01-04             1939.994                167.338  244.822431   
4   2009-01-05             1604.000                128.938  244.830778   
..         ...                  ...                    ...         ...   
690 2010-11-22             2041.536                142.354  240.197118   
691 2010-11-23             1577.536                137.450  240.575528   
692 2010-11-24             1796.248                132.460  240.030965   
693 2010-11-25             1431.164                116.128  241.536257   
694 2010-11-26             1488.104                120.826  240.291029   

     Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3     RR  \
0            4.149861          

In [10]:
new_df = daily_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 13, 8, 9, 10, 11, 12]]
print(new_df)

      DateTime  Global_active_power  Global_reactive_power     Voltage  \
0   2009-01-01             1406.692                112.668  242.836062   
1   2009-01-02             2327.510                204.730  242.704889   
2   2009-01-03             1913.496                174.912  243.681458   
3   2009-01-04             1939.994                167.338  244.822431   
4   2009-01-05             1604.000                128.938  244.830778   
..         ...                  ...                    ...         ...   
690 2010-11-22             2041.536                142.354  240.197118   
691 2010-11-23             1577.536                137.450  240.575528   
692 2010-11-24             1796.248                132.460  240.030965   
693 2010-11-25             1431.164                116.128  241.536257   
694 2010-11-26             1488.104                120.826  240.291029   

     Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
0            4.149861           765.0 

In [11]:
new_df.to_csv("test_daily.csv")